In [1]:
from dotenv import load_dotenv, find_dotenv
import os
import openai
from langchain.chat_models import ChatOpenAI
import json

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
Name_person="Abraham Lincoln"
user_prompt=f"Provide a birth place and date of birth of {Name_person}?"

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {
            "role": "user",
            "content": user_prompt,
        },
    ],
)


In [4]:
completion['choices'][0]['message']['content']

'Abraham Lincoln was born in a log cabin in Hodgenville, Kentucky, United States on February 12, 1809.'

In [5]:
function_descriptions = [
    {
        "name": "bio_info",
        "description": "Get the birth place and date of birth of a given person",
        "parameters": {
            "type": "object",
            "properties": {
                "Name": {
                    "type": "string",
                    "description": "person's name, e.g. Mahatma Gandhi"},
                "Date":{
                    "type":"string",
                    "description": "born on, e.g, February 12, 1920"},
                "Location":{
                    "type":"string",
                    "description": "Name of the city and country, e.g, St. Louis, MO, USA"}, 
            },
            "required": ["Name", "Date", "Location"],
        },
    }
]

In [6]:
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": user_prompt}],
    # Add function calling
    functions=function_descriptions,
    function_call="auto",  # specify the function call
)

In [7]:
output=completion['choices'][0]['message']['function_call']['arguments']

In [8]:
final_dictionary=json.loads(output)
final_dictionary

{'Name': 'Abraham Lincoln',
 'Date': 'February 12, 1809',
 'Location': 'Hodgenville, Kentucky, USA'}

In [9]:
completion

<OpenAIObject chat.completion id=chatcmpl-807OekdzVNf8XTzhmTEzBIWXJKWyR at 0x2093ef56660> JSON: {
  "id": "chatcmpl-807OekdzVNf8XTzhmTEzBIWXJKWyR",
  "object": "chat.completion",
  "created": 1695039124,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "bio_info",
          "arguments": "{\n  \"Name\": \"Abraham Lincoln\",\n  \"Date\": \"February 12, 1809\",\n  \"Location\": \"Hodgenville, Kentucky, USA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 119,
    "completion_tokens": 44,
    "total_tokens": 163
  }
}

In [10]:
def customized_output(Name, Date, Location):
    print(f"The person\'s name is {Name}, who was born in {Location}, and on {Date}.")

In [11]:
final_dictionary

{'Name': 'Abraham Lincoln',
 'Date': 'February 12, 1809',
 'Location': 'Hodgenville, Kentucky, USA'}

In [12]:
customized_output(*list(final_dictionary.values()))

The person's name is Abraham Lincoln, who was born in Hodgenville, Kentucky, USA, and on February 12, 1809.


In [13]:
json.loads(completion['choices'][0].message.function_call.arguments).get("Name")

'Abraham Lincoln'

In [14]:
Name = json.loads(completion['choices'][0].message.function_call.arguments).get("Name")
Date = json.loads(completion['choices'][0].message.function_call.arguments).get("Date")
Location=json.loads(completion['choices'][0].message.function_call.arguments).get("Location")
Name, Date, Location

('Abraham Lincoln', 'February 12, 1809', 'Hodgenville, Kentucky, USA')

In [15]:
print(Name, Date, Location)

Abraham Lincoln February 12, 1809 Hodgenville, Kentucky, USA


## LangChain Implementation

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains.openai_functions import create_structured_output_chain

In [17]:
llm = ChatOpenAI(model='gpt-3.5-turbo-0613')
chat_prompt_template = ChatPromptTemplate.from_template('{user_prompt}')

In [18]:
## without function calling

In [19]:
chain_no_functioncall=LLMChain(llm=llm, prompt=chat_prompt_template)

In [20]:
chain_no_functioncall.run(user_prompt=user_prompt)

'Abraham Lincoln was born on February 12, 1809, in Hodgenville, Kentucky, United States.'

In [23]:
user_prompt

'Provide a birth place and date of birth of Abraham Lincoln?'

In [24]:
langchain_function_descriptions = {
        "name": "bio_info",
        "description": "Get the birth place and date of birth of a given person",
        "type": "object",
        "properties": {
                "Name": {
                    "title":"Person's Name",
                    "type":"string",
                    "description":"person's name, e.g. Mahatma Gandhi"},
                "Date":{
                    "title":"Date of Birth",
                    "type":"string",
                    "description": "born on, e.g, February 12, 1920"},
                "Location":{
                    "title":"Birth Place",
                    "type":"string",
                    "description": "Name of the city and country, e.g, St. Louis, MO, USA"}, 
            }
    }

In [25]:
# Defining the LangChain chain object for function calling
chain = create_structured_output_chain(output_schema = langchain_function_descriptions,
                                       llm = llm,
                                       prompt = chat_prompt_template)

In [26]:
langchain_output=chain.run(user_prompt=user_prompt)

In [28]:
print(langchain_output)

{'Name': 'Abraham Lincoln', 'Date': 'February 12, 1809', 'Location': 'Hodgenville, Kentucky, USA'}
